# Obtén datos de la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Deberemos guardarlos en un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [3]:
import hashlib
import requests
import datetime
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

In [4]:
def hash_params(timestamp,priv_key,pub_key):
    """ Marvel API requires server side API calls to include
    md5 hash of timestamp + public key + private key """

    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

In [48]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

pub_key = 'dcfa67744d198a80776e92810ccdc2e0'
priv_key = '911ff403f58e6f8258d31e19892ce8d7b18c5578'

params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith':"M",
        # 'offset':100,
        'limit': 100
        };

url = 'http://gateway.marvel.com/v1/public/characters'

res = requests.get(url,params=params)

In [44]:
res

<Response [200]>

In [49]:
respuesta_json = res.json()
respuesta_json

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2024 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2024 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2024 MARVEL</a>',
 'etag': 'd4e030aebd41bccafb35893d14c3ea743c2d2e8c',
 'data': {'offset': 0,
  'limit': 100,
  'total': 157,
  'count': 100,
  'results': [{'id': 1011068,
    'name': 'M (Monet St. Croix)',
    'description': '',
    'modified': '2011-09-07T09:45:28-0400',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f',
     'extension': 'jpg'},
    'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1011068',
    'comics': {'available': 46,
     'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1011068/comics',
     'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/65072',
       'name': 'Generation X (2017) #8'},
      {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/65907',
       'name': 'Ge

In [50]:
respuesta_json.keys()

dict_keys(['code', 'status', 'copyright', 'attributionText', 'attributionHTML', 'etag', 'data'])

In [51]:
respuesta_json['data'].keys()

dict_keys(['offset', 'limit', 'total', 'count', 'results'])

In [52]:
respuesta_json['data']['total']

157

In [53]:
respuesta_json['data']['count']

100

In [54]:
respuesta_json['data']['results']

[{'id': 1011068,
  'name': 'M (Monet St. Croix)',
  'description': '',
  'modified': '2011-09-07T09:45:28-0400',
  'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f',
   'extension': 'jpg'},
  'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1011068',
  'comics': {'available': 46,
   'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1011068/comics',
   'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/65072',
     'name': 'Generation X (2017) #8'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/65907',
     'name': 'Generation X (2017) #86'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/66280',
     'name': 'Generation X (2017) #87'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/66565',
     'name': 'Generation X Vol. 2: Survival of the Fittest (Trade Paperback)'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/107618',
     'name': 'Un

In [18]:
respuesta_json['data']['results'][0].keys()

dict_keys(['id', 'name', 'description', 'modified', 'thumbnail', 'resourceURI', 'comics', 'series', 'stories', 'events', 'urls'])

In [25]:
respuesta_json['data']['results'][0]['thumbnail']['path'] + '.' + respuesta_json['data']['results'][0]['thumbnail']['extension']

'http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784.jpg'

In [21]:
marvel_dict = {"id": [1],
               "name": ["A"],
               "picture_url":["B"]}

pd.DataFrame(marvel_dict)

,id,name,picture_url
0,1,A,B


In [55]:
marvel_dict = {"id": [],
               "name": [],
               "picture_url":[]}

for personaje in respuesta_json['data']['results']:
    marvel_dict['id'].append(personaje.get("id"))
    marvel_dict['name'].append(personaje.get("name"))
    pic_url = personaje.get("thumbnail").get("path") + "." + personaje.get("thumbnail").get("extension")
    marvel_dict['picture_url'].append(pic_url)

df_marvel = pd.DataFrame(marvel_dict)
df_marvel.to_csv("data/marvel_personajes.csv")
df_marvel

,id,name,picture_url
0,1011068,M (Monet St. Croix),http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f.jpg
1,1011004,M.O.D.A.M.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
2,1011320,M.O.D.O.G.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1010726,M.O.D.O.K.,http://i.annihil.us/u/prod/marvel/i/mg/0/03/526034ba37851.jpg
4,1017321,M.O.D.O.K. (Iron Man 3 - The Official Game),http://i.annihil.us/u/prod/marvel/i/mg/c/00/5239c14fcddc0.jpg
5,1009418,Ma Gnuci,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
6,1009314,Mac Gargan,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
7,1010851,Mach IV,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
8,1010805,Machine Man,http://i.annihil.us/u/prod/marvel/i/mg/f/d0/4c003727804b4.jpg
9,1009411,Mad Thinker,http://i.annihil.us/u/prod/marvel/i/mg/b/c0/52740faf0d0fb.jpg


In [61]:
params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith':"M",
        'offset':100,
        'limit': 100
        };

url = 'http://gateway.marvel.com/v1/public/characters'

res = requests.get(url,params=params)
respuesta_json = res.json()

In [63]:
respuesta_json['data']['results']

[{'id': 1009446,
  'name': 'Mister Fear',
  'description': '',
  'modified': '1969-12-31T19:00:00-0500',
  'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available',
   'extension': 'jpg'},
  'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1009446',
  'comics': {'available': 4,
   'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1009446/comics',
   'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/20635',
     'name': 'Daredevil (1998) #105'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/8377',
     'name': 'Daredevil (1964) #373'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/8378',
     'name': 'Daredevil (1964) #374'},
    {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/22455',
     'name': 'Daredevil Saga (2008) #1'}],
   'returned': 4},
  'series': {'available': 3,
   'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1009446/series',
   

In [64]:
marvel_dict_2 = {"id": [],
               "name": [],
               "picture_url":[]}

for personaje in respuesta_json['data']['results']:
    marvel_dict_2['id'].append(personaje.get("id"))
    marvel_dict_2['name'].append(personaje.get("name"))
    pic_url = personaje.get("thumbnail").get("path") + "." + personaje.get("thumbnail").get("extension")
    marvel_dict_2['picture_url'].append(pic_url)

df_marvel_2 = pd.DataFrame(marvel_dict_2)
df_marvel_2
df_all = pd.concat([df_marvel, df_marvel_2])
df_all.to_csv("data/marvel_personajes_M.csv")
df_all

,id,name,picture_url
0,1011068,M (Monet St. Croix),http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f.jpg
1,1011004,M.O.D.A.M.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
2,1011320,M.O.D.O.G.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1010726,M.O.D.O.K.,http://i.annihil.us/u/prod/marvel/i/mg/0/03/526034ba37851.jpg
4,1017321,M.O.D.O.K. (Iron Man 3 - The Official Game),http://i.annihil.us/u/prod/marvel/i/mg/c/00/5239c14fcddc0.jpg
...,...,...,...
52,1010952,Mysterio (Francis Klum),http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
53,1009465,Mystique,http://i.annihil.us/u/prod/marvel/i/mg/5/03/5390dc2225782.jpg
54,1010953,Mystique (Age of Apocalypse),http://i.annihil.us/u/prod/marvel/i/mg/4/03/528d33b473550.jpg
55,1010954,Mystique (House of M),http://i.annihil.us/u/prod/marvel/i/mg/b/70/4c0033fbbdb54.jpg


In [ ]:
# obtener_personajes()

# almacenar_dataframe()

In [65]:
def obtener_personajes(endpoint, params):
    res = requests.get(endpoint, params=params)
    respuesta_json = res.json()
    return respuesta_json

def almacenar_dataframe(json):
    marvel_dict = {"id": [],
               "name": [],
               "picture_url":[]}

    for personaje in json['data']['results']:
        marvel_dict['id'].append(personaje.get("id"))
        marvel_dict['name'].append(personaje.get("name"))
        pic_url = personaje.get("thumbnail").get("path") + "." + personaje.get("thumbnail").get("extension")
        marvel_dict['picture_url'].append(pic_url)

    df = pd.DataFrame(marvel_dict)
    return df

In [ ]:
params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith':"M",
        'offset': 0,
        'limit': 100
        };

url = 'http://gateway.marvel.com/v1/public/characters'

respuesta = obtener_personajes(url, params)

df = almacenar_dataframe(respuesta)

if respuesta['total'] > if respuesta['count']:
    print("Faltan aún datos por obtener")
    params['offset'] += 100 
    respuesta = obtener_personajes(url, params['offset'])
else:


In [72]:
respuesta

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2024 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2024 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2024 MARVEL</a>',
 'etag': '82a9b15608486ab9d55ab3a4f8ba9f7d4726a408',
 'data': {'offset': 1200,
  'limit': 100,
  'total': 157,
  'count': 0,
  'results': []}}

In [74]:
params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith':"M",
        'offset': 0,
        'limit': 100
        };

url = 'http://gateway.marvel.com/v1/public/characters'

respuesta = obtener_personajes(url, params)

df = almacenar_dataframe(respuesta)

while len(df) < respuesta['data']['total']:
    print("Faltan aún datos por obtener")
    params['offset'] += 100 
    respuesta = obtener_personajes(url, params)

    df = pd.concat([df, almacenar_dataframe(respuesta)])
    
df.to_csv("data/marvel_personajes_" + params['nameStartsWith'] + '.csv')
df

Faltan aún datos por obtener


,id,name,picture_url
0,1011068,M (Monet St. Croix),http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f.jpg
1,1011004,M.O.D.A.M.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
2,1011320,M.O.D.O.G.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1010726,M.O.D.O.K.,http://i.annihil.us/u/prod/marvel/i/mg/0/03/526034ba37851.jpg
4,1017321,M.O.D.O.K. (Iron Man 3 - The Official Game),http://i.annihil.us/u/prod/marvel/i/mg/c/00/5239c14fcddc0.jpg
...,...,...,...
52,1010952,Mysterio (Francis Klum),http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
53,1009465,Mystique,http://i.annihil.us/u/prod/marvel/i/mg/5/03/5390dc2225782.jpg
54,1010953,Mystique (Age of Apocalypse),http://i.annihil.us/u/prod/marvel/i/mg/4/03/528d33b473550.jpg
55,1010954,Mystique (House of M),http://i.annihil.us/u/prod/marvel/i/mg/b/70/4c0033fbbdb54.jpg


In [78]:
def guardar_personajes_letra(letra):
    params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith': letra,
        'offset': 0,
        'limit': 100
        };

    url = 'http://gateway.marvel.com/v1/public/characters'
    print("Descargando personajes con la letra", letra)
    respuesta = obtener_personajes(url, params)

    df = almacenar_dataframe(respuesta)
    
    while len(df) < respuesta['data']['total']:
        print("Haciendo otra llamada para más personajes con la letra", letra)
        params['offset'] += 100 
        respuesta = obtener_personajes(url, params)

        df = pd.concat([df, almacenar_dataframe(respuesta)], ignore_index=True)
        
    df.to_csv("data/marvel_personajes_" + params['nameStartsWith'] + '.csv')
    df

In [84]:
lista_clase = ["Teresa", "Javi", "Ana", "Alicia", "Nico", "Santi", "Miguel"]

for persona in lista_clase:
    # if persona[0] not in archivos_carpeta:
    guardar_personajes_letra(persona[0])

Descargando personajes con la letra T
Descargando personajes con la letra J
Descargando personajes con la letra A
Descargando personajes con la letra A
Descargando personajes con la letra N
Descargando personajes con la letra S
Haciendo otra llamada para más personajes con la letra S
Haciendo otra llamada para más personajes con la letra S
Descargando personajes con la letra M
Haciendo otra llamada para más personajes con la letra M


In [ ]:
import os

In [82]:
os.getcwd()

'd:\\Bootcamp\\Carpeta_profesor\\2402_dsft_thebridge\\2-Data_Analysis\\3-Sources\\Web\\Práctica'

In [83]:
os.listdir()

['data', 'img', 'marvel_characters.ipynb']